In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
pd.__version__, np.__version__

('1.4.4', '1.24.3')

In [3]:
ano_susano = '2021'
data_files_path = os.path.join('.','Dados', ano_susano)
print(f'Nova {data_files_path=}')
tmp_path = os.path.join('/tmp')
pkl_source = 'LOCAL'

Nova data_files_path='.\\Dados\\2021'


In [ ]:
all_pkls = []

In [ ]:
def read_pkl(pkl_file):
    local_path = os.path.join(data_files_path, pkl_file)
    print(f'read_pkl: {local_path=}')
    gc_path = os.path.join(tmp_path, pkl_file)
    if pkl_source == 'LOCAL':
        df = pd.read_pickle(local_path)
    else:
        # download_blob(pkl_source, 'Dados/' + pkl_file, gc_path)
        df = pd.read_pickle(gc_path)
        os.remove(gc_path)
    all_pkls.append(pkl_file)
    return df

In [7]:
dtypes_EMA = {'CNES':str,
             'IBGE_Res':str,
             'IBGE_Estab': str,
             'IBGE_CAPITAL': str,
             'CO_GRUPO': str,
             'CO_GRUPO_SUBGRUPO': str}

In [9]:
def read_df(df_name):
    zip_file = df_name + '.zip'
    # local_path = os.path.join(data_files_path, pkl_file)
    local_path =  os.path.join(data_files_path, zip_file)
    print(f'{local_path=}')
    gc_path = os.path.join(tmp_path, zip_file)
    if pkl_source == 'LOCAL':
        df = pd.read_csv(local_path, dtype=dtypes_EMA)
        # df = pd.read_pickle(local_path)
    else:
        # download_blob(pkl_source, 'Dados/' + zip_file, gc_path)
        df = pd.read_csv(gc_path)
        os.remove(gc_path)
    return df

In [ ]:
ano_em_foco = int(ano_susano)

df_capitais = read_pkl('df_capitais.pkl')
UF_to_ibge_capital = df_capitais.set_index(['UF'])['IBGE_CAPITAL']
UF_to_nome_UF = df_capitais.set_index(['UF'])['NOME_UF']

In [ ]:
df_EMA = read_pkl('df_EMA.pkl')
# df_EMA contém os totais anuais para cada (Estabelecimento, Município, Alvo)

In [10]:
df_EMA = read_df('df_EMA')

local_path='.\\Dados\\2021\\df_EMA.zip'


In [ ]:
df_EMA_mês = read_pkl('df_EMA_mes.pkl')
# df_EMA_mês contém os valores mensais para cada (Estabelecimento, Município, Alvo, mês)

In [ ]:
df_M = read_pkl('df_M.pkl')
dict_M = df_M.set_index(['IBGE_Res']).T.to_dict('list')
# [NOME_MUN, SIGLA_UF, POP_MUN, POP_MUN_SUS, LAT, LONG, Qtd, Valor, Excesso] = dict_M[IBGE6_MUN]

In [ ]:
df_E = read_pkl('df_E.pkl')
CNES_to_Nome_Estab = df_E.set_index(['CNES'])['Nome_Estab']
dict_E = df_E.set_index(['CNES']).T.to_dict('list')
# [uf_estab, ibge_estab, nome_estab, abrev_estab, qtd_estab, valor_estab, excesso_estab]=dict_E['2077396']

In [12]:
df_A = read_df('df_A')
print(f'{df_A.columns=}')
dict_A = df_A.set_index(['Cod_Alvo']).T.to_dict('list')
# [nome_alvo, qtd, valor, excesso, abrev_alvo] = dict_A[cod_alvo]
Cod_Alvo_to_Alvo = df_A.set_index(['Cod_Alvo'])['Alvo']

local_path='.\\Dados\\2021\\df_A.zip'
df_A.columns=Index(['Cod_Alvo', 'Alvo', 'Qtd', 'Valor', 'Valor_Excesso_Estab',
       'Valor_Excesso_Estab_Min', 'Abrev_Alvo'],
      dtype='object')


In [ ]:
df_MA = read_pkl('df_MA.pkl')
print(f'******** df_MA.columns = {df_MA.columns}')
MA_to_excesso = df_MA.set_index(['IBGE_Res', 'Cod_Alvo'])['Excesso_MA']
MA_to_valor = df_MA.set_index(['IBGE_Res', 'Cod_Alvo'])['Valor_MA']
MA_to_qtd = df_MA.set_index(['IBGE_Res', 'Cod_Alvo'])['Qtd_MA']

In [13]:
df_EMA_g_A = df_EMA.groupby(['Cod_Alvo']).sum().reset_index()
df_EMA_g_A['Valor_Médio'] = df_EMA_g_A['Valor_EMA'] / df_EMA_g_A['Qtd_EMA']
df_EMA_g_A['Registro'] = df_EMA_g_A['Cod_Alvo'].str.slice(start=7)
df_EMA_g_A['Alvo'] = [Cod_Alvo_to_Alvo[cod_alvo] for cod_alvo in df_EMA_g_A['Cod_Alvo']]

In [14]:
df_EMA_g_A.to_csv(os.path.join(data_files_path, 'df_EMA_g_A.zip'))

In [ ]:
df_EMA_g_A = read_pkl('df_EMA_g_A')

In [ ]:
df_EMA_g_A.shape

In [ ]:
df_alvo_mês_limiar = read_pkl('df_alvo_mes_limiar.pkl')
A_mês_to_limiar = df_alvo_mês_limiar.set_index(['Cod_Alvo', 'Mês'])['Limiar']

In [ ]:
df_grupos = read_pkl('df_grupos.pkl')
dict_grupos = df_grupos.set_index(['CO_GRUPO']).T.to_dict('list')
# [nome_grupo] = dict_grupos['04']

In [ ]:
df_subgrupos = read_pkl('df_subgrupos.pkl')
dict_subgrupos = df_subgrupos.set_index(['CO_GRUPO_SUBGRUPO']).T.to_dict('list')
# [nomesubgrupo] = dict_subgrupos['0214']

In [ ]:
df_pop_UF = read_pkl('df_pop_UF.pkl')
dict_pop_UF = df_pop_UF.set_index(['SIGLA_UF']).T.to_dict('list')

In [ ]:
pop_Br = df_M['POP'].sum()
pop_SUS_Br = df_M['POP_SUS'].sum()

print(f'\n{all_pkls=}\n')

In [ ]:
import psutil
psutil.virtual_memory()